In [11]:
!pip3 install -Uqqq transformers einops accelerate langchain langchain-community bitsandbytes tiktoken sentence-transformers
!pip3 install -Uqqq langchain-chroma langchain_google_genai python-dotenv faiss-cpu langchain_huggingface streamlit


In [12]:
# Please Create a Gemini API Key using the following link
# https://ai.google.dev/gemini-api/docs/api-key

import os
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

In [13]:
import os
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI


def get_llm():

    print(f"Loading Environment Variables")
    load_dotenv()

    print(f"Loading LLM Module from Langchain")
    llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key = os.environ["GOOGLE_API_KEY"],temperature = 0.1)
    return llm

In [14]:
import os

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


abspath = os.path.abspath('run_chat.py')
dname = os.path.dirname(abspath)


def get_embedding_method():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embedding_function = HuggingFaceEmbeddings(model_name=model_name)
    return embedding_function


def get_docs():
    # Load documents locally as CSV
    loader = CSVLoader(f'{dname}/data/Customer-Support.csv')
    docs = loader.load()

    # Split document into text chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(docs)

    return docs


def get_embeddings():
    print("Loading Document")
    docs = get_docs()

    print("Loading Embedding Function")
    embedding_function = get_embedding_method()
    
    print("Creating Vector DB")
    db = Chroma.from_documents(docs, embedding_function)

    return db

In [15]:
from langchain.prompts import PromptTemplate


def get_prompt():
    
    prompt_templet = """
    You are a customer service chatbot

    {context}

    Answer the customer's questions only using the source data provided. 
    If you are unsure, say "I don't know, please call our customer support". 
    Use engaging, courteous, and professional language similar to a customer representative.
    Keep your answers concise.

    Generate the answer only for the given question. Do not generate extra text

    Question: {question}

    Answer:
    """

    prompt = PromptTemplate(
    template = prompt_templet,
    input_variables = ["context","question"]
    )
    
    return prompt

In [16]:
import os
import textwrap
import streamlit as st

import langchain

from langchain.chains import RetrievalQA

from src.langchain_llm import get_llm
from src.vectorDB_retriever import get_embeddings
from src.prompt_template import get_prompt


file = "vector_database_file"

def get_chain():
    
    llm = get_llm()
    embeddings = get_embeddings()
    retriever = embeddings.as_retriever(search_kwargs={"k": 1})
    prompt = get_prompt()
    
    chain = RetrievalQA.from_chain_type(
        
        llm = llm,
        chain_type = "stuff",
        retriever = retriever,
        input_key = "query",
        return_source_documents = True,
        chain_type_kwargs = {"prompt":prompt}
    
    )
    
    return chain


In [17]:
chain = get_chain()

Loading Environment Variables
Loading LLM Module from Langchain
Loading Document
Loading Embedding Function


/home/berserk/anaconda3/envs/test1/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Creating Vector DB


In [18]:
question = "I received a damaged product."
print(question)
response = chain.invoke(question)
print(response['result'])

I received a damaged product.
We apologize for the inconvenience. Can you please provide a photo of the damaged product so we can assist you further?


In [19]:
question = "Can I get a refund for my purchase?"
print(question)
response = chain.invoke(question)
print(response['result'])

Can I get a refund for my purchase?
We understand your concern. Please provide your order number and the reason for your refund request so we can assist you further.


In [20]:
question = "Can I add an item to an existing order?"
print(question)
response = chain.invoke(question)
print(response['result'])

Can I add an item to an existing order?
We'll do our best to help. Can you please provide your order number and the details of the item you'd like to add?
